<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

model = musicgen.MusicGen.get_pretrained('medium', device='cuda')
model.set_generation_params(duration=8)


  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-qyrfszoe/audiocraft_8363bcd7f09c40798074ba5a6cefbb2d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-qyrfszoe/audiocraft_8363bcd7f09c40798074ba5a6cefbb2d
  Resolved https://github.com/facebookresearch/audiocraft to commit 89baee7000318bec72cc589edbf6de4f948241ae
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.6 MB/s eta 0:00:00
  Prepari

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:127: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-medium
  warnings.warn(


state_dict.bin:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

try:
  import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import os
import soundfile as sf
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "df1b2ae9-f161-4e4d-9451-4d8d765ecbf5" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(audio_description: str):
    try:
        output = model.generate(
            descriptions=[audio_description],
            progress=True, return_tokens=True
        )

        # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
        generated_audio = output[0].cpu().numpy().astype('float32')
        generated_audio = generated_audio.squeeze()  # Remove extra dimensions
        sf.write('generated_audio.wav', generated_audio, 32000, subtype='PCM_16')  # Save as WAV file


        # after executing your custom code you send data back to the plugin like so ..
        await dawnet.results().add_file('generated_audio.wav', "wav")
        await dawnet.results().add_message("Returning the input file")
        await dawnet.results().send()

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


dawnet.set_token(token=dawnet_token)
dawnet.set_name("DAWNet Template")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method("dawnet_func", dawnet_func)

dawnet.connect_to_server()

Connected to ws://35.223.141.253:8765
Compute instance registered with the server.
METHODS REGISTERED: {'dawnet_func': <function dawnet_func at 0x786206e597e0>}
Compute instance registered with the server.
Sent contract for method dawnet_func
Compute instance registered with the server.
TEMP_DIR: /tmp/tmpy6os3awu
RAW_MSG: {"data": {"params": {"audio_description": {"type": "str", "value": "drum and bass"}}, "method_name": "dawnet_func"}, "type": "run_method", "token": "df1b2ae9-f161-4e4d-9451-4d8d765ecbf5", "message_id": "bdb70fad-ec79-4a44-b8b1-50e8822c5dce"}
PARSED_TYPE: run_method
RUN_METHOD: {'data': {'params': {'audio_description': {'type': 'str', 'value': 'drum and bass'}}, 'method_name': 'dawnet_func'}, 'type': 'run_method', 'token': 'df1b2ae9-f161-4e4d-9451-4d8d765ecbf5', 'message_id': 'bdb70fad-ec79-4a44-b8b1-50e8822c5dce'}
PARAMS: {'audio_description': 'drum and bass'}
run_method: self.method_registry: {'dawnet_func': <function dawnet_func at 0x786206e597e0>}
IS A COROUTINE
ST

KeyboardInterrupt: ignored

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
